# 位置が近すぎるラーメン屋を集約して点数の高い方を残す

In [4]:
import os
import pandas as pd

In [6]:
df = pd.read_csv(os.path.join(os.pardir, 'data', 'store.csv'))
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url
0,銀座 八五（はちごう）,3.93,東銀座駅,不明の為情報お待ちしております,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670537,139.770138,11：00~※スープがなくなり次第、営業終了とさせていただいております。,日曜日、土曜日（不定休）※不定休日の最新情報は、twitterをご覧ください。,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...
1,むぎとオリーブ 銀座本店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.669037,139.764321,11:00～20:00,月曜日（※月曜祝日の場合は火曜日もしくは水曜日休み）,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...
2,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[平日]11:00～20:00[土]11:00～19:00※売り切れ次第終了,日・祝,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671581,139.761402,"【月、火】11:30～15:30, 17:30～22:00【水〜金】11:30〜16:00,...",無休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...
4,麺処 銀笹（ぎんざさ）,3.78,築地市場駅,03-3543-0280,東京都中央区銀座8-15-2 藤ビル 1F,35.666225,139.763407,[月～土]11:15～15:00,日曜・祝日,18席 （４人掛けテーブル４卓、２人掛けテーブル１卓）,https://tabelog.com/tokyo/A1301/A130103/13119563/,https://tblg.k-img.com/resize/640x640c/restaur...


In [7]:
df['x'] = 1
df2 = pd.merge(df, df, on='x', how='inner', suffixes=['', '_'])[
    ['url', 'score', 'latitude', 'longitude', 
     'url_', 'score_', 'latitude_', 'longitude_']]
df2['distance'] = (df2['latitude'] - df2['latitude_']) ** 2 + (df2['longitude'] - df2['longitude_'])  ** 2
del df['x']
df2.head()

,url,score,latitude,longitude,url_,score_,latitude_,longitude_,distance
0,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,0.000000
1,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13164932/,3.78,35.669037,139.764321,0.000036
2,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130102/13046637/,3.78,35.675160,139.764343,0.000055
3,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13118141/,3.78,35.671581,139.761402,0.000077
4,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.93,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130103/13119563/,3.78,35.666225,139.763407,0.000064


In [8]:
# 距離が近い店舗の中で最大のスコアを持つ店舗を抽出
for i, threshold in enumerate([0.00001, 0.0001]):
    tmp = df2[df2['distance'] < threshold].groupby(['url', 'score'], as_index=False)[['score_']].max()
    tmp = tmp[tmp['score'] == tmp['score_']]
    df[f'summarize_{i}'] = (df['url'].isin(set(tmp['url']))).astype(int)
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url,summarize_0,summarize_1
0,銀座 八五（はちごう）,3.93,東銀座駅,不明の為情報お待ちしております,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670537,139.770138,11：00~※スープがなくなり次第、営業終了とさせていただいております。,日曜日、土曜日（不定休）※不定休日の最新情報は、twitterをご覧ください。,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...,1,1
1,むぎとオリーブ 銀座本店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.669037,139.764321,11:00～20:00,月曜日（※月曜祝日の場合は火曜日もしくは水曜日休み）,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
2,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[平日]11:00～20:00[土]11:00～19:00※売り切れ次第終了,日・祝,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671581,139.761402,"【月、火】11:30～15:30, 17:30～22:00【水〜金】11:30〜16:00,...",無休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
4,麺処 銀笹（ぎんざさ）,3.78,築地市場駅,03-3543-0280,東京都中央区銀座8-15-2 藤ビル 1F,35.666225,139.763407,[月～土]11:15～15:00,日曜・祝日,18席 （４人掛けテーブル４卓、２人掛けテーブル１卓）,https://tabelog.com/tokyo/A1301/A130103/13119563/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0


In [10]:
df.to_csv(os.path.join(os.pardir, 'data', 'store_summarize.csv'))